# 2.9.Stepik ML contest

Мы продолжаем предсказывать отток студентов с курса по Анализу данных в R. В этом модуле мы завершим подготовительный этап и доведем данные до состояния, когда можно обучить первый вариант модели!

## step 2

Итак, возвращаемся к нашей практической задаче - предсказание оттоков пользователей со степика.

В первом модуле мы закончили довольно важный и при этом довольно понятный этап для любой задачи, возникающей в machine-learning, аналитике, data-mining и т.д. У нас просто были какие-то сырые данные, и мы привели их к более понятному виду. Мы их смёрджили, предобработали, агрегировали. Мы как минимум получили понятную историю в этих данных, посмотрели, что всё с этими данными всё хорошо и с ними можно работать.

На самом деле ещё даже до применения моделей (особенно когда речь идёт о продуктовой аналитике и у нас есть цель скажем, разобраться почему пользователи дропаются) уже какие-то гипотезы можно получить, просто посмотрев на такой промежуточный агрегат. 

Допустим, уже сейчас можно изучить последние сессии дропнувшихся пользователей и убедиться в том, что один из таких частых сценариев - это когда человек натыкается на какую-то сложную задачу, очень долго подряд не может её решить и потом уже никогда к нашему курсу не возвращается.

В своё время, когда автор курса работал в степике, он первым делом пробежался по своим курсам и отредактировал довольно много задач. Особенно в курсе по R. Буквально в первом-втором уроке была сложная задача на работу с временными рядами и временные ряды до этого не обсуждались. Автору казалось, что самостоятельная работа слушателей с материалами по данной теме приведёт к положительному эффекту. На практике выяснилось, что огромное количество людей доходили до этой задачи, не могли её решить и больше никогда не появлялись на этом курсе.

И тут уже некоторый описательный анализ данных помог эту задачу изменить. на была перенесена в конец урока, выведена под звёздочку, было сказано, что необязательно решать её сейчас, поскольку можно пройти по курсу вперёд, а потом вернуться к её решению. И это в дальнейшем повлияло на результат.

Таким образом, некоторые интересные вещи ещё до моделирования, до обучения могут броситься в глаза.

Сейчас перед нами стоит задача продвинуться чуть дальше и привести данные к тому формату, когда уже можно будет обучать какие-то модели, делать интересные предсказания. 

Давайте переформулируем нашу задачу на язык машинного обучения. Наша цель как можно раньше предсказать, что пользователь уйдёт с курса "Анализ данных в R" и давайте для начала решим первую версию задачи. Для этого ответим на опрос - "можем ли мы, анализируя поведения пользователя за первые N дней, предсказать тот факт, что пользователь пройдёт курс до конца или дропнется и не наберёт нужное количество баллов?".

## Подгрузка предыдущих наработок

Подгружу в данный ноутбук наши ранее выполненные наработки.

In [117]:
import pandas as pd
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# reading datasets
events_data = pd.read_csv('event_data_train.zip', compression='zip')
submissions_data = pd.read_csv('submissions_data_train.zip', compression='zip')

# adding new columns
events_data['date'] = pd.to_datetime(events_data.timestamp, unit='s')
events_data['day'] = events_data.date.dt.date

submissions_data['date'] = pd.to_datetime(submissions_data.timestamp, unit='s')
submissions_data['day'] = submissions_data.date.dt.date

# calculating for each user count correct submimit
users_scores = submissions_data.pivot_table(index='user_id',
                        columns='submission_status',
                        values='step_id',
                        aggfunc='count',
                        fill_value=0).reset_index()

user_data = events_data.groupby('user_id', as_index=False) \
    .agg({'timestamp': 'max'}).rename(columns={
    'timestamp': 'last_timestamp'
})

now = 1526772811
drop_out_treshold = 30 * 24 * 60 * 60 # пороговое значение
user_data['is_gone_user'] = (now - user_data.last_timestamp) > drop_out_treshold

user_data = user_data.merge(users_scores, on='user_id', how='outer')
user_data = user_data.fillna(0)

users_events_data = events_data.pivot_table(index='user_id',
                        columns='action',
                        values='step_id', 
                        aggfunc='count', 
                        fill_value=0).reset_index()
user_data = user_data.merge(users_events_data, how='outer')

users_days = events_data.groupby('user_id').day.nunique()
users_days = events_data.groupby('user_id').day.nunique().to_frame().reset_index()
user_data = user_data.merge(users_days, how='outer')

user_data['passed_course'] = user_data.passed > 175

In [118]:
user_data.head()

,user_id,last_timestamp,is_gone_user,correct,wrong,discovered,passed,started_attempt,viewed,day,passed_course
0,1,1472827464,True,0.0,0.0,1,0,0,1,1,False
1,2,1519226966,True,2.0,0.0,9,9,2,10,2,False
2,3,1444581588,True,29.0,23.0,91,87,30,192,7,False
3,5,1499859939,True,2.0,2.0,11,11,4,12,2,False
4,7,1521634660,True,0.0,0.0,1,1,0,1,1,False


In [119]:
# Проверка, что данные не потеряны
user_data.user_id.nunique()

19234

In [120]:
events_data.user_id.nunique()

19234

Количество строк совпало, значит мы ничего не потеряли.

## step 3

На предыдущем этапе мы остановились на том, что предобработали данные и сделали сводную выгрузку users_data. 

Из сырых логов и разных источников мы свели все данные воедино. И, в принципе, можно наполнить эту табличку ещё несколькими важными колонками, но сейчас основная идея этой таблички была в том, что у нас были сырые логи, а теперь есть датафрейм users_data, в котором про каждого пользователя мы знаем:

+ его статус на текущий момент (правда ли, что он закончил курс успешно / дропнулся), + сколько он сделал попыток,   
+ когда его последний раз видели на курсе.

То есть пока ещё датафрейм, наверное, не очень применим для какого-то машинного обучения, хотя какие-то гипотезы можно уже и отсюда повытаскивать. Однако этот датафрейм нам точно потребуется ещё не раз, потому что, например, его можно использовать для разметки нашей выборки.

Теперь давайте подбираться к нашей первой задаче машинного обучения - попробуем, анализируя поведение пользователя за его первые несколько дней, после которых он начал курс, предсказать - правда ли, что он успешно закончит курсу или дропнется с него.

Для начала можно предположить, что мы можем использовать, скажем, первые 2-3 дня. Вообще, есл бы мы решали эту задачу по-настоящему - это как раз тот пример. когда у нас нет готового правильного ответа на вопрос "сколько первых дней нам нужно, чтобы предсказать уйдёт пользователь с курса или нет?". Поэтому можно попробовать разные пороги, нащупать какой-то оптимальный баланс времени и качества. Понятное дело, что если мы будем использовать первые две недели, то мы слишком поздно будем реагировать. Если мы будем пытаться предсказывать поведение пользователя за первые два часа с момента его начала работы с курсом - это, конечно, здорово, но возможно мы потеряем в точности.

Поэтому нужно экспериментировать и давайте начнём, например, с 3-х дней. Для начала можно в принципе проверить насколько это адекватно, потому что у нас есть данные про то сколько уникальных дней пользователь потратил на прохождение курса - колонка day. Можно посчитать медиану, предварительно отобрав только те наблюдения, у которых `passed_course = True`.

Обратите внимание, что это уже вектор True\False, поэтому можно просто передать его в качестве фильтра. Хотя это возможно не очень правильно, но так можно сделать.

добавим ещё одну колонку - когда пользователь первый раз появился на курсе.

In [ ]:
users_data[users_data.passed_course].day.median()

Мы видим, что медиана у нас 20 дней. Если построить гистограмму, то мы увидим типично скошенную метрику.

In [ ]:
users_data[users_data.passed_course].day.hist()

Кстати, это тоже интересное наблюдение - большинство метрик, как-то характеризующих взаимодействие пользователя с продуктом, очень часто распределены не нормально. Скажем, процент лайков, репостов в соцсетях, количество дней, затраченных пользователями на прохождение курса, число решённых задач и т.д. Скорее всего это будет ненормальное распределение (готовьтесь к этому). Такие классические симметричные распределения на самом деле в реальной практике на самом деле попадаются довольно редко.

Мы видим, что медиана 20, т.е. половина пользователей решает более 20 дней, поэтому 3 дня кажется адекватным порогом. Попробуем предсказать

## step 4

## step 5

## step 6

## step 8

## step 9

## step 10

## step 11

## step 12

## step 13